# Experiment 1, Model 1: Multilabel Classification of Linguistic Labels

In [1]:
# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For preprocessing
from gensim.models import FastText
from gensim import utils as gensim_utils

# For multilabel token classification
import sklearn.metrics
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

# For saving model
from joblib import dump,load

### 1. Create Word Embeddings

Train custom word embeddings on metadata descriptions from the University of Edinburgh Heritage Collections' Archives catalog.

* Data file: `descriptions_by_fonds`
* Date of harvesting: October 2020
* Harvesting and transformation code: [annot-prep/PreparationForAnnotation.ipynb](https://github.com/thegoose20/annot-prep/blob/main/PreparationForAnnotation.ipynb)

References:
* https://radimrehurek.com/gensim/models/fasttext.html
* https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py

In [2]:
dir_path = config.inf_data_path+"descriptions_by_fonds/"
file_list = os.listdir(dir_path)
print(len(file_list))

1079


In [3]:
class CorpusIterator:
    def __iter__(self):
        file_list = os.listdir(dir_path)
        for fonds_f in file_list:
            assert ".txt" in fonds_f, "All files should be Plaintext." 
            file_path = dir_path+fonds_f
            with utils.open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    # Lowercase the tokens
                    yield list(tokenize(line.lower()))   #list(tokenize(line))

Define the hyperparameters for the unsupervised training of the fastText model:

In [4]:
# Specify training architecture (default = "cbow" for Continuous Bag of Words)
training_arch = "cbow"  #"skipgram
# Specify the learning rate (default = 0.025)
alpha = 0.025
# Specify the training objective (default = "ns")
# losses = ["ns", "hs", "softmax"]
# loss = losses[0]
# Specify the number of negative words to sample for 'ns' training objective (default = 5)
negative = 5
# Specify the threshold for downsampling higher-frequency words (default = 0.001)
sample = 0.001
# Specify the word embeddings' dimensions
vector_dimensions = 100 #50 #300
# Specify the context window (default is 5) 
context_window = 5
# Specify the number of epochs (default is 5)
epochs = 5
# Specify the threshold of word occurrences (ignore words that occur less than specified number of times; default = 5)
min_count = 5
# Specify the minimum and maximum length of character ngrams (defaults are 3 and 6)
min_n = 2
max_n = 6  # if 0, no character n-grams (subword vectors) will be used
# Specify the number of buckets for hashing ngrams (default = 2000000) 
bucket = 2000000
# Sort vocabulary by descending frequency (default = 1)
sorted_vocab = 1
# Specify the number of threads to use (default = 12)
# threads = 12

In [5]:
embedding_model = FastText(
    alpha=alpha, negative=negative, sample=sample,
    vector_size=vector_dimensions, window=context_window, 
    epochs=epochs, min_count=min_count, min_n=min_n, 
    max_n=max_n, bucket=bucket, sorted_vocab=sorted_vocab
)

In [6]:
embedding_model.build_vocab(corpus_iterable=CorpusIterator())
total_examples = embedding_model.corpus_count

In [7]:
embedding_model.train(corpus_iterable=CorpusIterator(), total_examples=total_examples, epochs=epochs)

(7321545, 10119275)

Save the model:

In [8]:
file_name = "fasttext_{a}_{d}d.model".format(a=training_arch, d=vector_dimensions)
print(file_name)

fasttext_cbow_100d.model


In [9]:
embedding_model.save("models/"+file_name)

### 2. Data Preprocessing

In [94]:
# doc_data = "../data/token_clf_data/experiment_input/document_5fold.csv"
token_data = "../data/token_clf_data/experiment_input/token_5fold.csv"

In [95]:
text_col = "token"           # "description"

In [96]:
df = pd.read_csv(token_data, index_col=0)
df.head()

,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,fold
0,0,0,99999,0,Identifier,"(0, 10)",NN,O,Identifier,split4
1,0,0,99999,1,:,"(10, 11)",:,O,Identifier,split4
2,0,0,99999,2,AA5,"(12, 15)",NN,O,Identifier,split4
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,split2
4,1,1,99999,4,:,"(22, 23)",:,O,Title,split2


In [97]:
ling_labels = ["B-Generalization", "I-Generalization", "B-Gendered-Role", "I-Gendered-Role", "B-Gendered-Pronoun", "I-Gendered-Pronoun"]

In [98]:
labels_to_consider = ling_labels
col = "tag"

In [99]:
def implodeDataFrame(df, cols_to_groupby):
    cols_to_agg = list(df.columns)
    for col in cols_to_groupby:
        cols_to_agg.remove(col)
    agg_dict = dict.fromkeys(cols_to_agg, lambda x: x.tolist())
    return df.groupby(cols_to_groupby).agg(agg_dict).reset_index().set_index(cols_to_groupby)

def preprocessTokenData(df, col, label_list):
    initial_shape = df.shape
    # Change any tags not in label_list to "O"
    df_l = df.loc[df[col].isin(label_list)]
    df_o = df.loc[~df[col].isin(label_list)]
    df_o = df_o.drop(columns=[col])
    df_o.insert(len(df_o.columns), col, (["O"]*(df_o.shape[0])))
    df = pd.concat([df_l, df_o])
    df = df.sort_values(by="token_id")
    assert initial_shape == df.shape, "The DataFrame should have the same number of rows and columns after changing select column values."
    df = df.drop_duplicates()

    # Replace tags with labels, removing "B-" and "I-" from the start of the tags
    old_col = df[col]
    new_col = [tag[2:] if tag != "O" else tag for tag in old_col]
    df = df.drop(columns=[col])
    df.insert((len(df.columns)-2), col, new_col)
    
    # Drop unneeded columns and then get group by token, so there's one row per token and 
    # lists of tags for that token
    df = implodeDataFrame(df, ["description_id", "sentence_id", "token_id", "token", "pos", "field", "token_offsets"])
    df = df.reset_index()
    
    # Deduplicate tag lists and remove any "O" tags from lists with other values
    old_col = list(df[col])
    dedup_col = [list(set(value_list)) for value_list in old_col]
    assert len(old_col) == len(dedup_col), "The column should have the same number of rows."
    new_col = []
    for col_list in dedup_col:
        if ("O" in col_list) and (len(col_list) > 1):
            col_list.remove("O")
        new_col += [col_list]
    assert len(new_col) == len(old_col), "The column should have the same number of rows."
    df = df.drop(columns=[col])
    df.insert((len(df.columns)-2), col, new_col)
    
    return df  #.explode([col])  # one tag-token pair per row, tokens can repeat across rows

In [100]:
df = preprocessTokenData(df, col, labels_to_consider)
df.head()

,description_id,sentence_id,token_id,token,pos,field,token_offsets,tag,ann_id,fold
0,0,0,0,Identifier,NN,Identifier,"(0, 10)",[O],[99999],[split4]
1,0,0,1,:,:,Identifier,"(10, 11)",[O],[99999],[split4]
2,0,0,2,AA5,NN,Identifier,"(12, 15)",[O],[99999],[split4]
3,1,1,3,Title,NN,Title,"(17, 22)",[O],[99999],[split2]
4,1,1,4,:,:,Title,"(22, 23)",[O],[99999],[split2]


In [101]:
df[col].value_counts()

[O]                                   744728
[Gendered-Pronoun]                      3624
[Gendered-Role]                         3151
[Generalization]                        1808
[Gendered-Pronoun, Generalization]       107
[Gendered-Role, Generalization]          103
Name: tag, dtype: int64

### 3. Feature Extraction

In [102]:
def getFeatures(df, feature_cols=["token_id", "token"]):
    # Zip the features
    feature_data = list(zip(df[feature_cols[0]], df[feature_cols[1]]))
    
    # Make FastText feature matrix
    feature_list = [embedding_model.wv[token.lower()] for token_id,token in feature_data]
    return np.array(feature_list)

In [103]:
X = getFeatures(df)

### 4. Classifier Training

Define the five splits of the data to combine iteratively into training and test sets using five-fold cross-validation:

In [22]:
split_col = "fold"
splits = df[split_col].unique()
splits.sort()
print(splits)

['split0' 'split1' 'split2' 'split3' 'split4']


In [23]:
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]

In [24]:
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')
